In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
import numpy as np
from sklearn.cluster import KMeans

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
import cv2
from sklearn.metrics import accuracy_score
import glob

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall ,F1Score


import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
data_path = r"Combined_images"
image_list = []
file_names = []

# Iterate over each file in the specified directory
for filename in os.listdir(data_path):
    if filename.endswith(('.jpg', '.png', '.tif', '.JPG')):
        file_path = os.path.join(data_path, filename)
        img = cv2.imread(file_path)
        # Append the image and its filename to the lists
        image_list.append(img)
        file_names.append(filename)

In [3]:
# Function to extract the number from the image filename
def extract_number_from_filename(filename):
    # Assuming the number is after 'combined_' and before the first '.'
    return filename.split('_')[1].split('.')[0]

In [4]:
clean_file_names = []
for name in file_names:
    clean_file_names.append(extract_number_from_filename(name))

In [5]:
# Sample data (replace these with your actual data)
images = image_list
image_names = clean_file_names

# Function to read the CSV and create a dictionary of image names to labels
def read_csv_to_dict(csv_file):
    df = pd.read_csv(csv_file)

    # Extract image names and labels
    image_names = df.iloc[:, 0].values
    labels = df.iloc[:, -1].values

    # Create a list of image-label pairs
    image_label_array = np.array([[name, label] for name, label in zip(image_names, labels)])
    
    # Convert the NumPy array to a dictionary
    image_label_dict = {row[0]: row[1] for row in image_label_array}
    return image_label_dict


# Function to create the list of images and their corresponding labels
def create_image_label_list(images, image_names, image_label_dict):
    image_label_list = []
    for image, name in zip(images, image_names):
        if name in image_label_dict:
            label = image_label_dict[name]
            image_label_list.append([name,image, label])
    return image_label_list

# Main execution
csv_file_path = 'image_labels.csv'  # Replace with your actual CSV file path
image_label_dict = read_csv_to_dict(csv_file_path)
image_label_list = create_image_label_list(images, image_names, image_label_dict)

In [6]:
print(f'Number of images: {len(image_label_list)}')

Number of images: 625


In [7]:
df = pd.read_csv('image_labels.csv', names=['image_num','b','c','label'])

In [15]:
df['label'].value_counts().to_frame()

,count
label,
Misr,251
al-Mansuriyah,223
m.,104
al-Mahdiyah,40
(al-Mansuriyah),10
Filastin,9
al-Mansuriyah?,8
uncertain,5
(Atarablus),4


In [16]:
two_largest_list = []
for instance in image_label_list:
    if instance[2]=='Misr' or instance[2]=='al-Mansuriyah'or instance[2]=='(al-Mansuriyah)':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
        two_largest_list.append(instance)

In [17]:
print(f'Number of images: {len(two_largest_list)}')

Number of images: 452


In [11]:
# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

In [12]:
X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 for x in Y])

In [13]:
X.shape

(452, 300, 600, 3)

In [14]:
def print_ratios_one_input(Y):
    # Assuming y_train_first is your training labels ndarray
    unique, counts = np.unique(Y, return_counts=True)

    # Combine the unique labels with their counts
    label_counts = dict(zip(unique, counts))

    print(label_counts)

In [15]:
def print_ratios_Two_inputs(Y_train,Y_val):
    # Assuming y_train_first is your training labels ndarray
    unique, counts = np.unique(Y_train, return_counts=True)

    # Combine the unique labels with their counts
    label_counts_train = dict(zip(unique, counts))

    # Print the count of each label
    # Assuming y_train_first is your training labels ndarray
    unique, counts = np.unique(Y_val, return_counts=True)

    # Combine the unique labels with their counts
    label_counts_test = dict(zip(unique, counts))
    print("train",label_counts_train)
    print("val",label_counts_test)

In [16]:
print_ratios_one_input(Y)

{0: 235, 1: 217}


In [17]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

In [18]:
print_ratios_Two_inputs(y_train_first,y_test)

train {0: 188, 1: 173}
val {0: 47, 1: 44}


In [19]:
y_test = to_categorical(y_test, num_classes=2)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

In [21]:
print_ratios_Two_inputs(y_train,y_val)

train {0: 150, 1: 138}
val {0: 38, 1: 35}


In [22]:
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

In [59]:

model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

In [60]:
model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 300, 600, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 150, 300, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 150, 300, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 75, 150, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 360000)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │      46,080,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,085,474 (175.80 MB)

 Trainable params: 46,085,474 (175.80 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [62]:
history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.4629 - loss: 20.9588 - val_accuracy: 0.5205 - val_loss: 0.7538
Epoch 2/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 739ms/step - accuracy: 0.5301 - loss: 1.2465 - val_accuracy: 0.5205 - val_loss: 0.7277
Epoch 3/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 695ms/step - accuracy: 0.5523 - loss: 0.6917 - val_accuracy: 0.4795 - val_loss: 0.6955
Epoch 4/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 690ms/step - accuracy: 0.4485 - loss: 0.7101 - val_accuracy: 0.4795 - val_loss: 0.6907
Epoch 5/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 687ms/step - accuracy: 0.5295 - loss: 0.6950 - val_accuracy: 0.5205 - val_loss: 0.6930
Epoch 6/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 680ms/step - accuracy: 0.5404 - loss: 0.7034 - val_accuracy: 0.5205 - val_loss: 0.6929
Epoch 7/18
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 678ms/step - accuracy: 0.4828 - loss: 0.6985 - val_accuracy: 0.5205 - val_loss: 0.7036
Epoch 7: early stopping


In [27]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.3587842583656311 Accuracy: 0.8681318759918213


In [36]:
# Define the model
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(300, 600, 3), padding='same', name='conv2d_4'))
model.add(MaxPooling2D((2, 2), name='max_pooling2d_4'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='conv2d_5'))
model.add(MaxPooling2D((2, 2), name='max_pooling2d_5'))
model.add(Flatten(name='flatten_2'))
model.add(Dense(128, activation='relu', name='dense_4'))
model.add(Dropout(0.2, name='dropout_2'))
model.add(Dense(2, activation='softmax', name='dense_5'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=18, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Save the model
model.save('my_model.h5')

# Load the model
model = load_model('my_model.h5')

# Make a prediction to build the model layers
_ = model.predict(np.expand_dims(X_test[0], axis=0))

# Function to compute Grad-CAM
def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads

    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = np.zeros(output.shape[0:2], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam.numpy(), (img_array.shape[2], img_array.shape[1]))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())
    
    return heatmap

# Function to display Grad-CAM
def display_gradcam(img, heatmap):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlay = heatmap + np.float32(img)
    overlay = overlay / np.max(overlay)
    plt.figure(figsize=(10, 10))
    plt.imshow(overlay)
    plt.show()

# Select an image from the test set
img = X_test[0]
img_array = np.expand_dims(img, axis=0)

# Get Grad-CAM heatmap for the last convolutional layer
layer_name = 'conv2d_5'  # Use the correct last conv layer name
heatmap = get_gradcam(model, img_array, layer_name)

# Display Grad-CAM
display_gradcam(img, heatmap)

Epoch 1/18


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32,), output.shape=(32, 2)

In [79]:
three_largest_list = []
for instance in image_label_list:
    if instance[2]=='Misr' or instance[2]=='al-Mansuriyah' or instance[2]=='(al-Mansuriyah)' or instance[2]=='m.':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
            print(instance[2])
        three_largest_list.append(instance)

In [80]:
len(three_largest_list)

543

In [134]:
# Convert resized_image_list to numpy array
X = np.array([row[1] for row in three_largest_list])
Y = [row[2] for row in three_largest_list]

In [82]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 25,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    validation_split = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [135]:
X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 if x == 'al-Mansuriyah' else 2 for x in Y])

In [139]:
Y

In [140]:
print_ratios_one_input(Y)

{0: 235, 1: 217, 2: 91}


In [141]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42,stratify=Y)

In [142]:
print_ratios_Two_inputs(y_train,y_test)

train {0: 188, 1: 173, 2: 73}
val {0: 47, 1: 44, 2: 18}


In [144]:
y_test = to_categorical(y_test, num_classes=3)
print(y_test)

In [145]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=42,stratify=y_train)

In [146]:
print_ratios_Two_inputs(y_train_2,y_val)

train {0: 150, 1: 138, 2: 59}
val {0: 38, 1: 35, 2: 14}


In [148]:
y_train_2 = to_categorical(y_train_2, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)
print(y_train_2)
print(y_val)

In [29]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [113]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer for 3 classes
model.add(Dense(3, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 360000)            0         
                                                                 
 dense_12 (Dense)            (None, 128)              

In [114]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True,verbose=1)

In [115]:
# Train the model
history = model.fit(
     X_train_2, y_train_2,
#     datagen.flow(X_train, y_train, batch_size=32),
    epochs=18,  # Increase the number of epochs to allow for more training
    validation_data=(X_val, y_val)
#     callbacks=[early_stopping]
)


Epoch 1/18
11/11 [==============================] - 14s 1s/step - loss: 26.1985 - accuracy: 0.4121 - val_loss: 3.0150 - val_accuracy: 0.1609
Epoch 2/18
11/11 [==============================] - 11s 1s/step - loss: 1.6203 - accuracy: 0.3948 - val_loss: 1.0177 - val_accuracy: 0.5287
Epoch 3/18
11/11 [==============================] - 11s 969ms/step - loss: 1.0555 - accuracy: 0.5418 - val_loss: 1.0411 - val_accuracy: 0.5747
Epoch 4/18
11/11 [==============================] - 11s 962ms/step - loss: 0.9670 - accuracy: 0.5965 - val_loss: 0.8285 - val_accuracy: 0.6897
Epoch 5/18
11/11 [==============================] - 10s 930ms/step - loss: 0.6530 - accuracy: 0.7493 - val_loss: 0.8456 - val_accuracy: 0.7011
Epoch 6/18
11/11 [==============================] - 10s 955ms/step - loss: 0.3538 - accuracy: 0.8934 - val_loss: 0.6479 - val_accuracy: 0.8046
Epoch 7/18
11/11 [==============================] - 10s 936ms/step - loss: 0.2011 - accuracy: 0.9481 - val_loss: 0.6626 - val_accuracy: 0.7586
Epoc

In [116]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.8091941475868225 Accuracy: 0.7431192398071289


    We want to improve our accuracy.
    I order to do that we will try to find why our model is having difficulties by checking how well it does between all types of coins in pairs
   

In [155]:
# make a list of m. and al-Mansuriyah

two_largest_list = []
for instance in image_label_list:
    if instance[2]=='m.' or instance[2]=='al-Mansuriyah'or instance[2]=='(al-Mansuriyah)':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
        two_largest_list.append(instance)

In [157]:
len(two_largest_list)

308

In [160]:
# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

In [161]:
X = X/255.0
Y = np.array([0 if x == 'm.' else 1 for x in Y])

In [162]:
print_ratios_one_input(Y)

{0: 91, 1: 217}


In [171]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

In [172]:
print_ratios_Two_inputs(y_train_first,y_test)

train {0: 73, 1: 173}
val {0: 18, 1: 44}


In [174]:
y_test = to_categorical(y_test, num_classes=2)
print(y_test)

In [175]:
X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

In [176]:
print_ratios_Two_inputs(y_train,y_val)

train {0: 58, 1: 138}
val {0: 15, 1: 35}


In [178]:
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
print(y_train)
print(y_val)

In [185]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

In [186]:
model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_12 (Flatten)        (None, 360000)            0         
                                                                 
 dense_24 (Dense)            (None, 128)             

In [187]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [188]:
history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val)
#     callbacks=[early_stopping]
)

Epoch 1/18
7/7 [==============================] - 10s 1s/step - loss: 22.9564 - accuracy: 0.6224 - val_loss: 5.8904 - val_accuracy: 0.7000
Epoch 2/18
7/7 [==============================] - 8s 1s/step - loss: 3.6090 - accuracy: 0.6224 - val_loss: 1.5570 - val_accuracy: 0.7000
Epoch 3/18
7/7 [==============================] - 6s 919ms/step - loss: 0.9939 - accuracy: 0.6735 - val_loss: 0.7001 - val_accuracy: 0.4000
Epoch 4/18
7/7 [==============================] - 6s 914ms/step - loss: 0.6403 - accuracy: 0.7194 - val_loss: 0.6234 - val_accuracy: 0.7000
Epoch 5/18
7/7 [==============================] - 7s 933ms/step - loss: 0.5848 - accuracy: 0.7041 - val_loss: 0.6277 - val_accuracy: 0.7000
Epoch 6/18
7/7 [==============================] - 6s 840ms/step - loss: 0.5869 - accuracy: 0.7194 - val_loss: 0.6005 - val_accuracy: 0.6800
Epoch 7/18
7/7 [==============================] - 6s 840ms/step - loss: 0.5233 - accuracy: 0.7704 - val_loss: 0.6044 - val_accuracy: 0.6800
Epoch 8/18
7/7 [========

In [189]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 1.3793977499008179 Accuracy: 0.6774193644523621


In [192]:
# make a list of m. and Misr

two_largest_list = []
for instance in image_label_list:
    if instance[2]=='m.' or instance[2]=='Misr':
        two_largest_list.append(instance)

len(two_largest_list)

# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

X = X/255.0
Y = np.array([0 if x == 'm.' else 1 for x in Y])

print_ratios_one_input(Y)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

print_ratios_Two_inputs(y_train_first,y_test)

y_test = to_categorical(y_test, num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

print_ratios_Two_inputs(y_train,y_val)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

{0: 91, 1: 235}
train {0: 73, 1: 187}
val {0: 18, 1: 48}
train {0: 58, 1: 150}
val {0: 15, 1: 37}


In [195]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1,restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_31 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 360000)            0         
                                                                 
 dense_30 (Dense)            (None, 128)             

In [196]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.17107048630714417 Accuracy: 0.939393937587738


In [198]:
# make a list of al-Mahdiyah  and Misr

two_largest_list = []
for instance in image_label_list:
    if instance[2]=='al-Mahdiyah' or instance[2]=='Misr':
        two_largest_list.append(instance)

len(two_largest_list)

# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 for x in Y])

print_ratios_one_input(Y)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

print_ratios_Two_inputs(y_train_first,y_test)

y_test = to_categorical(y_test, num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

print_ratios_Two_inputs(y_train,y_val)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

{0: 235, 1: 37}
train {0: 187, 1: 30}
val {0: 48, 1: 7}
train {0: 149, 1: 24}
val {0: 38, 1: 6}


In [199]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1,restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_32 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_33 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_16 (Flatten)        (None, 360000)            0         
                                                                 
 dense_32 (Dense)            (None, 128)             

In [200]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.31666919589042664 Accuracy: 0.8545454740524292


In [201]:
# make a list of al-Mahdiyah  and al-Mansuriyah

two_largest_list = []
for instance in image_label_list:
    if instance[2]=='al-Mahdiyah' or instance[2]=='al-Mansuriyah'or instance[2]=='(al-Mansuriyah)':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
        two_largest_list.append(instance)

len(two_largest_list)

# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

X = X/255.0
Y = np.array([0 if x == 'al-Mahdiyah' else 1 for x in Y])

print_ratios_one_input(Y)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

print_ratios_Two_inputs(y_train_first,y_test)

y_test = to_categorical(y_test, num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

print_ratios_Two_inputs(y_train,y_val)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

{0: 37, 1: 217}
train {0: 30, 1: 173}
val {0: 7, 1: 44}
train {0: 24, 1: 138}
val {0: 6, 1: 35}


In [202]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1,restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_17 (Flatten)        (None, 360000)            0         
                                                                 
 dense_34 (Dense)            (None, 128)             

In [203]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.41825732588768005 Accuracy: 0.8627451062202454


In [204]:
# make a list of al-Mahdiyah  and al-Mansuriyah and Misr

three_largest_list = []
for instance in image_label_list:
    if instance[2]=='al-Mahdiyah' or instance[2]=='al-Mansuriyah' or instance[2]=='(al-Mansuriyah)' or instance[2]=='Misr':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
        three_largest_list.append(instance)

len(three_largest_list)

# Convert resized_image_list to numpy array
X = np.array([row[1] for row in three_largest_list])
Y = np.array([row[2] for row in three_largest_list])

X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 if x == 'al-Mansuriyah' else 2 for x in Y])

print_ratios_one_input(Y)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)

print_ratios_Two_inputs(y_train_first,y_test)

y_test = to_categorical(y_test, num_classes=3)

X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42, stratify=y_train_first)

print_ratios_Two_inputs(y_train,y_val)

y_train = to_categorical(y_train, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)

{0: 235, 1: 217, 2: 37}
train {0: 188, 1: 173, 2: 30}
val {0: 47, 1: 44, 2: 7}
train {0: 150, 1: 138, 2: 24}
val {0: 38, 1: 35, 2: 6}


In [209]:
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer for 3 classes
model.add(Dense(3, activation='softmax'))

model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1,restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=18,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_42 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_43 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_21 (Flatten)        (None, 360000)            0         
                                                                 
 dense_42 (Dense)            (None, 128)             

In [210]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.7337673902511597 Accuracy: 0.7857142686843872
